## Imports

In [6]:
import sys, os
import numpy as np
import pandas as pd
import pretty_midi as pm
import time

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.io.record_midi import MidiInput
from src.io import output, input

## Ratings

In [7]:
stability = (6, 2, 4, 2, 5, 4, 2, 5, 2, 4, 2, 4, 6) # index = pitch relative to root note
proximity = (24, 36, 32, 25, 20, 16, 12, 9, 6, 4, 2, 1, 0.25) # index = interval in semitones; mobility factor (* 2/3 for repetition) is already included at index 0
direction_continuation = (6, 20, 12, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0) # index = previous interval in semitones
direction_reversal = (0, 0, 0, 0, 0, 6, 12, 25, 36, 52, 75, 75, 75) # index = previous interval in semitones

## Expectancy Formula

In [8]:
def calc_expectancy_rating_for_pitch(candidate: int, prev: int, prevprev: int, single_ratings = False):
    
    prev_interval = abs(prev-prevprev)
    current_interval = abs(candidate-prev)
    prev_direction = np.sign(prev-prevprev)
    current_direction = np.sign(candidate-prev)

    continuation = current_direction != 0 and current_direction == prev_direction

    s = stability[candidate]
    p = proximity[current_interval]
    d = direction_continuation[prev_interval] if continuation else direction_reversal[prev_interval]

    if current_direction == 0:
        d /= 3

    if prev == 11 and candidate == 9:
        s = 6

    if single_ratings:
        return s, p, d
    else:
        return (s * p) + d

## Record Starting Notes

In [9]:
rec = MidiInput()

RuntimeError: MidiInAlsa::initialize: error creating ALSA sequencer client object.

In [5]:
midi = rec.recordNotes(2)
notes = midi.instruments[0].notes
rec.close_port()

[IO] NOW RECORDING ...
[REC] 67 70 
[IO] recording successful.
[IO] Input port "MPK Mini Mk II 0" closed.


## Prediction Settings

In [6]:
number_of_generated_notes = 8
root_offset = 60
randomized_choice = True

## Normalize Starting Notes

As no rhythm and articulation will be generated, all generated notes will have a velocity of *100* and a length of *0.5 seconds*.

For the sake of consistency, the input notes will be normalized to the same values.

Pitch values are shifted so that the root note has a pitch of 0.

In [7]:
for note in notes:
    note.velocity = 100
    note.pitch -= root_offset
    note.start = notes.index(note)/2
    note.end = note.start+0.5

In [8]:
# debug
notes

[Note(start=0.000000, end=0.500000, pitch=7, velocity=100),
 Note(start=0.500000, end=1.000000, pitch=10, velocity=100)]

## Calculation of Expectancy Ratings and Prediction

In [9]:
expectancy_ratings = []
predictions = []

t1 = time.time()

for note in range(0, number_of_generated_notes):
    prev_pitch = notes[-1].pitch
    prevprev_pitch = notes[-2].pitch
    ratings_for_note = []

    # calculate ratings for all 12 possible pitches
    for pitch in range(0,13):
        e = calc_expectancy_rating_for_pitch(pitch, prev_pitch, prevprev_pitch)
        ratings_for_note.append(e)

    # append expectancy ratings to list
    expectancy_ratings.append(ratings_for_note)

    # select pitch with highest rating
    if not randomized_choice:
        prediction = ratings_for_note.index(max(ratings_for_note))
    else:
        normalized_ratings = [float(rating)/sum(ratings_for_note) for rating in ratings_for_note]
        prediction = np.random.choice(range(0,13), 1, p=normalized_ratings)[0]

    predictions.append(prediction)
    print('prediction ' + str(note) + ': ' + str(prediction))

    # create and append predicted note to midi
    predicted_note = pm.Note(100, prediction, note/2+1, note/2+1.5)
    notes.append(predicted_note)

t2 = time.time()
print('execution time: ' + str(t2-t1))

prediction 0: 11
prediction 1: 12
prediction 2: 12
prediction 3: 11
prediction 4: 8
prediction 5: 10
prediction 6: 11
prediction 7: 4
execution time: 0.004999637603759766


In [10]:
table = pd.DataFrame(data=expectancy_ratings)
table['prediction'] = predictions
table

,0,1,2,3,4,5,6,7,8,9,10,11,12,prediction
0,12.0,8,24,18,60,64,40,125,64.0,144,48.0,150.0,198.0,11
1,6.0,4,16,12,45,48,32,100,50.0,192,72.0,96.0,236.0,12
2,1.5,2,8,8,30,36,24,80,40.0,100,64.0,144.0,144.0,12
3,1.5,2,8,8,30,36,24,80,40.0,100,64.0,144.0,144.0,11
4,26.0,24,36,32,65,68,52,120,70.0,212,92.0,96.0,216.0,8
5,42.0,24,54,38,106,106,70,186,48.0,144,64.0,100.0,120.0,10
6,12.0,8,24,18,60,64,40,125,64.0,144,48.0,156.0,204.0,11
7,6.0,4,16,12,45,48,32,100,50.0,192,72.0,96.0,236.0,4


## Revoke Pitch Normalization

In [11]:
for note in notes:
    note.pitch += root_offset

## Plott Piano Roll

In [12]:
output.saveMidiFile(midi, '../midi/margulis-cache.mid')
midi = input.loadMidiFile('../midi/margulis-cache.mid')
output.pianoRoll(midi)

[IO] file saved to:


d:\FH\Masterarbeit\Project\output\margulis-cache.mid

Loading BokehJS ...

In [13]:
#debug
midi.instruments[0].notes

[Note(start=0.000000, end=0.500000, pitch=67, velocity=100),
 Note(start=0.500000, end=1.000000, pitch=70, velocity=100),
 Note(start=1.000000, end=1.500000, pitch=71, velocity=100),
 Note(start=1.500000, end=2.000000, pitch=72, velocity=100),
 Note(start=2.000000, end=2.500000, pitch=72, velocity=100),
 Note(start=2.500000, end=3.000000, pitch=71, velocity=100),
 Note(start=3.000000, end=3.500000, pitch=68, velocity=100),
 Note(start=3.500000, end=4.000000, pitch=70, velocity=100),
 Note(start=4.000000, end=4.500000, pitch=71, velocity=100),
 Note(start=4.500000, end=5.000000, pitch=64, velocity=100)]

In [14]:
output.synthesizeMidi(midi)